In [ ]:
%run '../util/neptune.py'

In [ ]:
g = neptune.graphTraversal(neptune_endpoint=os.environ['NEPTUNE_READER_ENDPOINT'])

## Find people with the most followers

The following query calculates the vertex degree for every node in the graph, based on incoming 'follows' relationships. It then prints the top ten most followed people in the graph and their degree (number of followers).

Because this is a full-graph query, this may take several seconds to complete.

In [ ]:
mostFollowersIds = (g.V().
  project('v','degree').
    by(id).
    by(inE('follows').count()).
  order().by(select('degree'),decr).
  select('v','degree').limit(10).
  toList())



print(json.dumps(mostFollowersIds, indent=2))

## Find people who follow the most people

The following query calculates the vertex degree for every node in the graph, based on outgoing 'follows' relationships. It then prints the top ten people with the most outgoing 'follows' relationships.

Because this is a full-graph query, this may take several seconds to complete.

In [ ]:
mostFollowingIds = (g.V().
  project('v','degree').
    by(id).
    by(outE('follows').count()).
  order().by(select('degree'),decr).
  select('v','degree').limit(10).
  toList())

print(json.dumps(mostFollowingIds, indent=2))

## Recommend new people to follow

In the next query we'll take the person following the greatest number of people (from the results of the query above), and recommend new people they may wish to follow.

The query looks for people followed by the people the user follows who are not currently followed by the user. For example, given user A, if A follows B, B follows C, but A does not follow C, then C will be of interest to us. 

Having found people at depth 2 who are not currently followed by the user, but who are indirectly connected to the user by way of the people the user follows at depth 1, the query then counts the number of paths that link each person at depth 2 to the user, and sorts the results based on the number of paths. For example, if there are 5 different paths to C (by way of 5 different people at depth 1), and 7 different paths to D (by way of 7 different people at depth 1), then D will come higher in the results than C.

In [ ]:
personId = mostFollowingIds[0]['v']

recommendations = (g.V(personId).as_('user'). 
  # depth 1
  choose(
    select('user').outE('follows').count().as_('user-count').is_(lt(100)), # if user follows <100 people
      out('follows'),                                                      # follow all edges
      choose(
        select('user-count').is_(lt(1000)), # else if user follows <1000 people
          out('follows').coin(0.5),         # follow half the edges
          out('follows').coin(0.1)          # otherwise, follow one tenth the edges
      )
  ).as_('f').aggregate('following').
  # depth 2
  choose(
    select('f').outE('follows').count().as_('f-count').is_(lt(1000)), # if total number people followed by people user follows < 1000
      out('follows'),                                                 # follow all edges
      choose(
        select('f-count').is_(lt(10000)), # if total number people followed by people user follows < 10,000
          out('follows').coin(0.5),       # follow half the edges
          out('follows').coin(0.1)        # otherwise, follow one tenth the edges
      )
  ).where(neq('user')).where(without('following')).
  group().by().by(count()).unfold().
  project('v', 'count').
  by(keys).
  by(values).
  order().by(select('count'), decr).
  limit(10).
  project('firstName', 'lastName', 'id', 'count').
  by(select('v').by('firstName')).
  by(select('v').by('lastName')).
  by(select('v').by('id')).
  by(select('count')).
  toList())

print(json.dumps(recommendations, indent=2))